In [2]:
# Data from chapter, Imbalanced Learn - Not even 1% win the lottery
from sklearn.datasets import fetch_openml
data = fetch_openml(data_id=1220)

df = pd.DataFrame(
    data['data'],
    columns=data['feature_names']
)[
    [
        'user_id',
        'ad_id'
    ]
].astype(int)


df['clicked'] = pd.Series(data['target']).astype(int) 

In [3]:
df['clicked'].mean()

0.16841894462801643

In [4]:
df.head(4)

,user_id,ad_id,clicked
0,0,8343295,0
1,562934,20017077,1
2,11621116,21348354,0
3,8778348,20366086,0


In [5]:
df.sample(n=500).groupby(['user_id', 'ad_id']).max().reset_index().pivot('user_id', 'ad_id', 'clicked').fillna(0)

ad_id,1192697,1211044,1211676,1355299,1544201,1918047,2376704,2511991,2635400,2710813,...,21968587,22002401,22039139,22072959,22073320,22073611,22096779,22102490,22129963,22161302
user_id,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
186,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
275,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
747,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
982,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23400971,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23552582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23562382,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Loading Data

In [14]:
from surprise.dataset import Dataset
from surprise import Reader

reader = Reader(rating_scale=(0, 1))
dataset = Dataset.load_from_df(df, reader)

# Train/Test Split

In [17]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(dataset, test_size=0.25)

# K-Fold Cross Validation 

In [110]:
from surprise.model_selection import cross_validate

def predict_evaluate(recsys, dataset, name='Algorithm'):
    scores = cross_validate(recsys, dataset, measures=['RMSE', 'MAE'], cv=4, verbose=False)
    print(
        'Testset Avg. MAE: {:.2f} & Avg. RMSE: {:.2f} [{}]'.format(
            scores['test_mae'].mean(),
            scores['test_rmse'].mean(),
            name
        )
    )

# Random Recommender

In [111]:
from surprise import AlgoBase

class RandomRating(AlgoBase):

    def __init__(self, p=0.5):
        self.p = p
        AlgoBase.__init__(self)

    def estimate(self, u, i):
        return np.random.binomial(n=1, p=self.p, size=1)[0]


recsys = RandomRating(p=0.16)
predict_evaluate(recsys, dataset, 'RandomRating')

Testset Avg. MAE: 0.28 & Avg. RMSE: 0.53 [RandomRating]


In [112]:
from surprise.prediction_algorithms.random_pred import NormalPredictor

recsys = NormalPredictor()
predict_evaluate(recsys, dataset, 'NormalPredictor')

Testset Avg. MAE: 0.33 & Avg. RMSE: 0.47 [NormalPredictor]


In [113]:
from surprise.prediction_algorithms.knns import KNNBasic

recsys = KNNBasic(verbose=False)
predict_evaluate(recsys, dataset, 'KNNBasic')

Testset Avg. MAE: 0.28 & Avg. RMSE: 0.38 [KNNBasic]


In [131]:
from surprise.prediction_algorithms.baseline_only import BaselineOnly

recsys = BaselineOnly(verbose=False)
predict_evaluate(recsys, dataset, 'BaselineOnly')

Testset Avg. MAE: 0.27 & Avg. RMSE: 0.37 [BaselineOnly]


In [115]:
from surprise.prediction_algorithms.matrix_factorization import SVD

recsys = SVD(verbose=False)
predict_evaluate(recsys, dataset, 'SVD')

Testset Avg. MAE: 0.27 & Avg. RMSE: 0.37 [SVD]


# Grid Search Cross Validation

In [163]:
from surprise.model_selection import GridSearchCV
from surprise.prediction_algorithms.baseline_only import BaselineOnly

param_grid = {
    'p': [0.01, 0.05, 0.1, 0.15, 0.2, 0.25]
}

gscv = GridSearchCV(RandomRating, param_grid, measures=['rmse', 'mae'], cv=4, n_jobs=-1)
gscv.fit(dataset)

print('Best MAE:', gscv.best_score['mae'].round(2))
print('Best RMSE:', gscv.best_score['rmse'].round(2))
print('Best Params', gscv.best_params['rmse'])

Best MAE: 0.18
Best RMSE: 0.42
Best Params {'p': 0.01}


In [164]:
from surprise.model_selection import GridSearchCV
from surprise.prediction_algorithms.baseline_only import BaselineOnly

param_grid = {
    'bsl_options': {
        'method':['als'],
        'n_epochs': [10, 50, 100],
        'reg_u': [5, 15, 25],
        'reg_i': [5, 10, 20],
        
    },
    'verbose': [False]
}

gscv = GridSearchCV(BaselineOnly, param_grid, measures=['rmse', 'mae'], cv=4, n_jobs=-1)
gscv.fit(dataset)

print('Best MAE:', gscv.best_score['mae'].round(2))
print('Best RMSE:', gscv.best_score['rmse'].round(2))
print('Best Params', gscv.best_params['rmse'])

Best MAE: 0.27
Best RMSE: 0.37
Best Params {'bsl_options': {'method': 'als', 'n_epochs': 50, 'reg_u': 25, 'reg_i': 10}, 'verbose': False}


In [85]:
from surprise.prediction_algorithms.baseline_only import BaselineOnly
from surprise import accuracy

recsys = BaselineOnly()
recsys.fit(trainset)
predictions = recsys.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

Estimating biases using als...
RMSE: 0.3688


0.3688410769535897

In [89]:
from surprise.prediction_algorithms.matrix_factorization import SVD
from surprise import accuracy

recsys = SVD(
    n_factors=20,
    n_epochs=100,
    verbose=False,
)
recsys.fit(trainset)
predictions = recsys.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

RMSE: 0.3749


0.3748614956923788

In [51]:
from surprise.prediction_algorithms.knns import KNNBasic
from surprise import accuracy

recsys = KNNBasic()
recsys.fit(trainset)
predictions = recsys.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.3792


0.37920859429757436

In [52]:
from surprise.prediction_algorithms.knns import KNNWithMeans
from surprise import accuracy

recsys = KNNWithMeans()
recsys.fit(trainset)
predictions = recsys.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.3785


0.3785244871388106

In [53]:
from surprise.prediction_algorithms.knns import KNNBaseline
from surprise import accuracy

recsys = KNNBaseline()
recsys.fit(trainset)
predictions = recsys.test(testset)

# Then compute RMSE
accuracy.rmse(predictions)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.3727


0.3726847981061818